# 自定义层

In [2]:
import torch
import torch.nn.functional as F

from torch import nn

### 不带参数的层
构建一个**没有任何参数**的自定义层

In [4]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()


layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件**合并到构建更复杂的模型中**
- 因为存储精度的原因，仍然可能会看到一个非常小的非零数

In [5]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.2282e-09, grad_fn=<MeanBackward0>)

### 带参数的图层
实现**自定义版本的全连接层**，需要两个参数——一个用来表示**权重**，一个用来表示**偏置项**
- `dense = MyLinear(5, 3) `得到了一个`MyLinear`的实例

In [6]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.rand(units))

    def forward(self, X):
        linear = torch.mm(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3) 
dense.weight        

Parameter containing:
tensor([[ 0.3745,  0.8938, -1.3152],
        [-0.2532, -0.2514, -2.2754],
        [ 0.6998, -0.2958, -0.5224],
        [ 0.0787,  1.4705,  0.4032],
        [ 1.2067,  0.2011,  0.4960]], requires_grad=True)

**使用自定义层直接执行正向传播计算**

In [8]:
dense(torch.rand(2, 5))

tensor([[2.4242, 2.3293, 0.0000],
        [1.6959, 2.4471, 0.0000]])

**使用自定义的层构建模型**

In [9]:
net = nn.Sequential(MyLinear(64, 8), nn.Linear(8, 1))
net(torch.rand(2, 64))

tensor([[ 0.5184],
        [-0.9763]], grad_fn=<AddmmBackward0>)

### 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。
* 在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

### 练习
1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
- `.diagonal()`是pytorch中用于提取矩阵或张量**对角线元素**的方法
    - 取矩阵乘法得到的**矩阵的对角线元素**

In [10]:
class TensorReduction(nn.Module):
    def __init__(self, dim1, dim2):
        super().__init__()
        # 定义一个可训练的权重参数，维度为(dim2, dim1, dim1)
        self.weight = nn.Parameter(torch.rand(dim2, dim1, dim1))

    def forward(self, X):
        # 初始化一个全零张量，大小为(X.shape[0], self.weight.shape[0])
        Y = torch.zeros(X.shape[0], self.weight.shape[0])
        for k in range(self.weight.shape[0]):
            # 计算temp = X @ weight[k] @ X^T
            temp = X @ self.weight[k] @ X.T
            # 取temp的对角线元素，存入Y[:, k]
            Y[:, k] = temp.diagonal()
        return Y

layer = TensorReduction(10, 5)
X = torch.rand(2, 10)
layer(X).shape

torch.Size([2, 5])

2. 设计一个返回输入数据的傅立叶系数前半部分的层。

- `torch.fft`模块用于计算张量的傅立叶变换及操作
    - `torch.fft.fft(input, dim=-1)`计算输入张量的离散傅里叶变换
    - `torch.fft.ifft(input, dim=-1)`计算傅里叶变换的逆变换
    - ...
- `X = X[..., :X.shape[-1] // 2]`其中`...`表示**除最后一个维度以外**的所有维度不变

In [19]:
import torch

from torch import nn, fft

class FourierLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        # 先傅里叶变换
        X = fft.fftn(X)
        X = X[..., :X.shape[-1] // 2]
        return X

X = torch.rand(20, 100, 100)
net = FourierLayer()
net(X).shape
        

torch.Size([20, 100, 50])

# 读写文件

### 加载和保存张量
- 对单个张量，可以直接调用`load`和`save`来读写，都**要求提供一个名称**

In [20]:
import torch

from torch import nn
from torch.nn import functional as F

In [22]:
x = torch.arange(4)
torch.save(x, 'x-file')   # 保存在当前目录下

x1 = torch.load('x-file', weights_only=True)
x1

tensor([0, 1, 2, 3])

存储一个**张量*列表***，将其读回内存

In [24]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files', weights_only=True)
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取**从字符串映射到张量的*字典***

In [27]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict1 = torch.load('mydict', weights_only=True)
mydict1

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 加载和保存模型参数
- 模型本身可以包含任意代码，故模型本身难以序列化
- 需要**用代码生成架构**，然后**从磁盘加载参数**

In [38]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

X = torch.rand(2, 20)
net = MLP()
Y = net(X)

将**模型的参数储存为一个叫做‘mlp.params’的文件**

In [39]:
torch.save(net.state_dict(), 'mlp.params')

#### **实例化了原始多层感知机模型的一个备份**，直接读取文件中存储的参数
- 从存储的文件中**加载了一个多层感知机的参数**
- 然后**将这些参数加载到一个新的`MLP`模型实例中**，实现模型的恢复

- `.load_state_dict()`：方法会将模型的所有参数用加载的参数文件**替换**，故**无需重新初始化模型参数**

In [40]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params', weights_only=True))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例**具有相同的模型参数**，在输入相同的`X`时，两个实例的**计算结果应该相同**

In [42]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* **保存架构必须*在代码中*完成**，而不是在参数中完成。

### 练习
1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
- **加速模型训练**：可以闭麦每次重新训练模型时需重复计算之前已经计算过的权重和偏置
- **节省内存空间**：比保存完整的模型文件更加节省内存空间
- **便于共享和复现**
- **便于调试和分析**

2. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

In [43]:
import torch

from torch import nn
from torch.nn import functional as F

In [45]:
# 原先的就用前面定义好的 `MLP`

class MLP_new(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')
clone = MLP_new()
clone.hidden.load_state_dict(torch.load('mlp.hidden.params', weights_only=True))
print(clone.hidden.weight == net.hidden.weight)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


3. 如何同时保存网络架构和参数？需要对架构加上什么限制？
- **不包含动态构造的层**，即不包含任何随机性质的操作，如dropout层的随机丢弃率应该是固定的
- 网络架构需要再定义模型的文件中**可用**：pytorch会在加载模型时查找构建该模型的类定义
    - 在加载完整模型的文件中，**引入构建模型的类定义**，如`from my_model_file import MLP`
- **注意**：在加载文件时显示指定`weights_only=False`

In [48]:
import torch

from torch import nn
from torch.nn import functional as F

net = MLP()
torch.save(net, 'model_complete.pt')
net_loaded = torch.load('model_complete.pt', weights_only=False)
net_loaded.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)